In [1]:
# Carrega as bibliotecas
from nltk.classify import NaiveBayesClassifier, MaxentClassifier, SklearnClassifier
import csv
from sklearn.svm import LinearSVC

In [2]:
# Função para criar uma lista de palavras dada uma frase
def divide(dados):    
    dados_new = []
    for palavra in dados:
        palavra_filter = [i.lower() for i in palavra.split()]
        dados_new.append(palavra_filter)
    return dados_new

In [3]:
# Função para criar a lista de palavras e suas frequências
def bag_of_words(palavras):
    return dict([(palavra, palavras.count(palavra)) for palavra in palavras])

In [4]:
# Treina os classificadores
def treina_classificadores():
    posdados = []
    with open('train_EPTC_POA_v3nbal_1.data', 'rt', encoding='utf-8') as myfile:    
        reader = csv.reader(myfile, delimiter=',')
        for val in reader:
            posdados.append(str(val[0]))               
    negdados = []
    with open('train_EPTC_POA_v3nbal_0.data', 'rt', encoding='utf-8') as myfile:    
        reader = csv.reader(myfile, delimiter=',')
        for val in reader:
            negdados.append(val[0])                   
    neudados = []
    with open('train_EPTC_POA_v3nbal_2.data', 'rt', encoding='utf-8') as myfile:    
        reader = csv.reader(myfile, delimiter=',')
        for val in reader:
            neudados.append(val[0])                         
    negfeats = [(bag_of_words(f), 'neg') for f in divide(negdados)]
    posfeats = [(bag_of_words(f), 'pos') for f in divide(posdados)]
    neufeats = [(bag_of_words(f), 'neu') for f in divide(neudados)]
    treino = negfeats + posfeats+ neufeats
    #'Maximum Entropy'
    classificadorME = MaxentClassifier.train(treino, 'GIS', trace=0, encoding=None, labels=None, gaussian_prior_sigma=0, max_iter = 1)
    #SVM
    classificadorSVM = SklearnClassifier(LinearSVC(), sparse=False)
    classificadorSVM.train(treino)
    # Naive Bayes
    classificadorNB = NaiveBayesClassifier.train(treino)
    return ([classificadorME,classificadorSVM,classificadorNB])

In [5]:
# Função para retornar a classificação das frases
def classifica(sentencas, classificadores):
    ret = []                        
    for s in sentencas:
        c = divide([s])
        feats= bag_of_words(c[0])
        classificacao = []
        classificacao.append(classificadores[1].classify(feats))
        classificacao.append(classificadores[2].classify(feats))
        classificacao.append(classificadores[0].classify(feats))
        ret.append(classificacao)
    return ret

In [6]:
# Treina os classificadores
classificadores = treina_classificadores()

In [7]:
# Define as sentenças
sentences = ['Fluxo muito congestionado.', \
            'Não use o celular ao volante, 80% da sua atenção é desviada.']

print(sentences)

['Fluxo muito congestionado.', 'Não use o celular ao volante, 80% da sua atenção é desviada.']


In [8]:
# Realiza a classificação
classifica(sentences, classificadores)

[['neg', 'neu', 'neg'], ['neu', 'neu', 'neu']]